The usual preliminaries....


In [1]:
pip install sklearn


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")['train']

Found cached dataset imdb (C:/Users/thoma/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train_data = []
train_data_labels = []
for item in imdb_dataset:
  train_data.append(item['text'])
  train_data_labels.append(item['label'])

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word',max_features=10000,lowercase=True)
features = vectorizer.fit_transform(train_data).toarray()

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(features,train_data_labels,train_size=0.75,random_state=123)

We will use three models, Multinomial Naive Bayes, Random Forests and Logistic Regression to do the classification. Create the models.

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
model_nb = MultinomialNB()
model_lr = LogisticRegression()
model_rf = RandomForestClassifier()

Train the models.

In [10]:
model_nb = model_nb.fit(X=X_train,y=y_train)
model_lr = model_lr.fit(X=X_train,y=y_train)
model_rf = model_rf.fit(X=X_train,y=y_train)

C:\Users\thoma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test the models on the validation set.

In [11]:
y_pred_nb = model_nb.predict(X_val)
y_pred_lr = model_lr.predict(X_val)
y_pred_rf = model_rf.predict(X_val)

Now let's calculate the accuracy of the models' predictions on the validation set.

In [12]:
from sklearn.metrics import accuracy_score, confusion_matrix
print("Naive Bayes", accuracy_score(y_val,y_pred_nb))
print(confusion_matrix(y_val,y_pred_nb))
print()
print("Logistic Regression", accuracy_score(y_val,y_pred_lr))
print(confusion_matrix(y_val,y_pred_lr))
print()
print("Random Forest", accuracy_score(y_val,y_pred_rf))
print(confusion_matrix(y_val,y_pred_rf))


Naive Bayes 0.83664
[[2642  443]
 [ 578 2587]]

Logistic Regression 0.87216
[[2661  424]
 [ 375 2790]]

Random Forest 0.84192
[[2605  480]
 [ 508 2657]]


In [ ]:
#I will explore the use of sentiment lexicons to improve classification accuracy
#with lexicons words are given a polarity score this number can be postive or negative.
#A lexicon based approach differnt to ML based approach for classification
##There are two types of lexicon approaches: Dictionary-based and Corpus-based
#I will look at a dictionary based approach for classififcation
#I will look at lexicon dictionaries like SentiWordNet WordNetLemmatizer etc

In [ ]:
#Looking at different lexicon dictionaries
!pip install nltk

In [17]:
#Implementing sentiment lexiocn dictionaries on a single sentence
import nltk
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
nltk.download('punkt')
from nltk.tokenize import word_tokenize
list(swn.senti_synsets('slow'))
sentence='It was a really good day'
from nltk.tag import pos_tag
token = nltk.word_tokenize(sentence)
after_tagging = nltk.pos_tag(token)
print (token)
print (after_tagging)
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
sentiment = 0.0
tokens_count = 0
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
for word, tag in after_tagging:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            print(swn_synset)
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
print (sentiment)#Postitve sentiment score of 1.375

['It', 'was', 'a', 'really', 'good', 'day']
[('It', 'PRP'), ('was', 'VBD'), ('a', 'DT'), ('really', 'RB'), ('good', 'JJ'), ('day', 'NN')]
<truly.r.01: PosScore=0.625 NegScore=0.0>
<good.a.01: PosScore=0.75 NegScore=0.0>
<day.n.01: PosScore=0.0 NegScore=0.0>
1.375


[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
#This is then tested on the training data loaded above. Not on all the training data as would take too long to run.
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
predicts = []
for i in train_data[0:2000]:
    token = nltk.word_tokenize(i)
    after_tagging = nltk.pos_tag(token)
    #print (token)
    #print (after_tagging)
    sentiment = 0.0
    tokens_count = 0
    lemmatizer = WordNetLemmatizer()
    for word, tag in after_tagging:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
           # print(swn_synset)
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
    if sentiment > 0:
        predicts.append(1)#if the overall sentiment is postive append 1
    else:
        predicts.append(0)#if overall sentiment isn't postitve append 0
   # print ("sent", sentiment)
print(accuracy_score(train_data_labels[0:2000], predicts))#0.63, 0.66308
#Prints the accuracy on the first 2000 training reviews
#The accuracy here is lower than with the ML models above.

0.63


print(X_Val)